## Metodologia 1

Essa metodologia segue a seguinte definição:

- Definir um script para setar pontos por iteração, para que, a cada novo ponto adicionado, seja melhorado o resultado da segmentação. Foi notado em experimentos manuais usando ViT-H que, ao colocar um ponto positivo na região central da fácies que se deseja segmentar, o modelo consegue capturar boa parte da região alvo, porém acaba "vazando" para regiões vizinhas. Assim, o algoritmo abaixo define o processo da adição dos pontos:

### Algoritmo 1
1. Escolha uma fácie para ser segmentada.
2. Defina a quantidade de pontos que serão usados.
3. Calcule o ponto médio dessa região e adicione um ponto na região que represente o meio.
4. Calcule a diferença entre a área da fácie real com a predita pelo modelo.
5. Se a diferença sobrar região para a predição, então o próximo ponto deverá ser negativo. Caso contrário, ou seja, se a diferença sobrar para a área real, então o próximo ponto deverá ser positivo.
6. Volte para o passo 3 e subtraia o total de pontos possíveis.
7. Para cada passo, salve o iou de cada ponto e o iou acumulado.
8. Calcule a área sob a curva (AOC) da segmentação usando os valores de IoU acumulado por iteração.

### Algoritmo 2
1. Escolha uma fácie para ser segmentada.
2. Defina a quantidade de pontos que serão usados.
3. Calcule o ponto médio dessa região.
4. Realize os seguintes passos para escolher o próximo ponto:
    1. Defina uma janela m x m ao redor da região central calculada no passo 3).
    2. Teste todos os pontos candidatos (positivo e negativo em cada quadrante) dentro dessa janela (m² experimentos).
    3. Escolha o ponto (positivo ou negativo) que maximize o IoU.
    4. Adicione o ponto escolhido ao modelo e atualize a predição.
5. Volte para o passo 3 e subtraia o total de pontos possíveis.
6. Para cada passo, salve o iou de cada ponto e o iou acumulado.
7. Calcular também o delta IoU para avaliar o ganho marginal obtido com a adição do ponto
8. Calcule a área sob a curva (AOC) da segmentação usando os valores de IoU acumulado por iteração.

- OBS:
1. Para cada algoritmo:
    - Salvar visualmente as predições intermediárias (máscaras) em cada iteração pode ajudar a entender como os pontos positivos/negativos impactam a segmentação.
    - Plotar as curvas de IoU acumulado ao longo das iterações.

Após isso, fazer a seleção de X amostras e requisitar pessoas para realizar a segmentação das fácies da seguinte forma:
- Dado uma seção, segmente as F fácies as segas usando N pontos.
- Dado uma seção e sua label, olhe uma vez a label e segmente as F fácies com os mesmos N pontos.